In [ ]:
!pip install spams
import numpy as np
import spams
batchSize=720
param = {'lambda1':0.2,'numThreads' : -1,'mode' : 1}
alpha_trans1=[]
x_train= np.load('../../../datasets/ORL/ORLdataforSb.npy')

alpha_trans1=[]
for l in range (0,len(x_train),batchSize):
    xtr=x_train[l:l+batchSize].T
    alpha=np.zeros((xtr.shape[1],xtr.shape[1]))
    for i in range (xtr.shape[1]):
        x=xtr[:,i].reshape(400,1)
        if i==0:
            xf=xtr[:,i+1:].reshape(400,xtr.shape[1]-1)
            alpha[0,0]=0
            tmp = spams.lasso(x,D =xf ,return_reg_path = False,**param)
            #print(tmp)
            tmp=tmp.toarray()
            #print(tmp.shape)
            tmp=tmp.reshape(xtr.shape[1]-1)
            alpha[1:,i] = tmp
        else:  
            dictionary= np.concatenate([xtr[:,:i],xtr[:,i+1:]],axis=1)
            xf=dictionary.reshape(400,xtr.shape[1]-1)
            tmp = spams.lasso(x,D = xf,return_reg_path = False,**param)
            #print(tmp)
            tmp=tmp.toarray()
            tmp=tmp.reshape(xtr.shape[1]-1)
            alpha[:i,i]=tmp[:i]
            alpha[i,i]=0
            alpha[i+1:,i]=tmp[i:]
        alpha_trans1.append(alpha.T)
    alpha_trans1=np.array(alpha_trans1)
    alpha_trans1=alpha_trans1.astype('float32')
    return alpha_trans1
np.save('Hs_orl.npy',alpha_trans1)